In this notebook reservoir sampling is being implemented. This is a method to randomly sample from a stream of data of unknown length.                          
 
Generally random sampling methods rely on knowing the number of items in the population, or on having random access to the storage.  But with a very large amount of data, for example too large to hold in memory, or data which is streamed and not stored in its entirety, this might not be the case.

Reservoir sampling gets around this issue by holding $k$ items in a priority queue sorted according to a random tag value as the stream of data is processed.

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/chembl22/chembl_22_clean_1576904_sorted_std_final.smi


In [2]:
from pathlib import Path
import random
import heapq

In [3]:
datafile = Path('/kaggle/input/chembl22/chembl_22_clean_1576904_sorted_std_final.smi')

In [4]:
class ReservoirSample:
    @property
    def fileline(self):
        l = self.file.readline()
        # once EOF has been reached, readline returns empty string
        if len(l) == 0:
            return None
        # tag each item with a random value
        return (random.random(), l)
        
    def __init__(self, path, k=5):
        self.heap = []
        self.file = path.open()
        while True:
            s = self.fileline
            if s is None:
                break
            if len(self.heap) < k:
                heapq.heappush(self.heap, s)
            else:
                # add the new item to the heap, then remove the item with the 
                # smallest tag value
                heapq.heappushpop(self.heap, s)
        self.file.close()
        
    @property
    def sample(self):
        return [h[1] for h in self.heap]
        
    def __del__(self):
        if not self.file.closed:
            self.file.close()

In [5]:
s = ReservoirSample(datafile)

The heap with the randomly generated tag values:

In [6]:
s.heap

[(0.9999983777034297, 'CCOC(=O)c1cc(CI)[nH]n1\tCHEMBL346266\n'),
 (0.9999992155910636,
  'COc1ccc(cc1)S(=O)(=O)c1ccc(NC(=O)c2ccccc2Cl)cc1C#N\tCHEMBL1552950\n'),
 (0.9999989447676946,
  'CCCCNc1c(cc2C(=O)N(CCCC)C(=O)c3cccc1c23)N(=O)=O\tCHEMBL2234768\n'),
 (0.9999998988631276,
  'COc1cc2nccc(Oc3ccc4N(CCOc4c3)C(=O)Nc3ccc(Cl)cc3)c2cc1OC\tCHEMBL270548\n'),
 (0.9999992247537781,
  'CCOC(=O)NC(CC(=O)c1ccccc1)C(=O)NC(C(C)C)C(=O)NC(C)C(=O)NC(CC(C)C)C(N)=O\tCHEMBL311211\n')]

The random sample of the data stream:

In [7]:
s.sample

['CCOC(=O)c1cc(CI)[nH]n1\tCHEMBL346266\n',
 'COc1ccc(cc1)S(=O)(=O)c1ccc(NC(=O)c2ccccc2Cl)cc1C#N\tCHEMBL1552950\n',
 'CCCCNc1c(cc2C(=O)N(CCCC)C(=O)c3cccc1c23)N(=O)=O\tCHEMBL2234768\n',
 'COc1cc2nccc(Oc3ccc4N(CCOc4c3)C(=O)Nc3ccc(Cl)cc3)c2cc1OC\tCHEMBL270548\n',
 'CCOC(=O)NC(CC(=O)c1ccccc1)C(=O)NC(C(C)C)C(=O)NC(C)C(=O)NC(CC(C)C)C(N)=O\tCHEMBL311211\n']